In [1]:
import re
import time
import json
import uuid
import redis
import datetime
import pandas as pd

from sqlalchemy import create_engine
from selenium import webdriver

In [8]:
best_seller_url = {
    'taobao':'https://s.taobao.com/search?initiative_id=tbindexz_20170306&ie=utf8&spm=a21bo.2017.201856-taobao-item.2&sourceId=tb.index&search_type=item&ssid=s5-e&commend=all&imgfile=&q=连衣裙&suggest=history_1&_input_charset=utf-8&wq=&suggest_query=&source=suggest&sort=sale-desc',
    'tmall':'https://list.tmall.com/search_product.htm?spm=a220m.1000858.1000724.4.62613ec2vteq0U&q=连衣裙&sort=d&style=g&from=.list.pc_1_searchbutton#J_Filter'
}

source_goods_url = {
    '17':'https://gz.17zwd.com/sks.htm?so=羽绒服&cateid=&display_name=&f_akmk=&color=&size=&mid=&sv=&fid=&range=&ord=&fs=&ml=&video=&image=&recomme=&pstart=&pend='
}

In [2]:
import configparser
cf = configparser.ConfigParser()
cf.read("config/crawl.cfg", encoding='utf-8')

['config/crawl.cfg']

In [27]:
a = [[1,2],[3,4]]
b = [[5,6]]
a.extend(b)
a

[[1, 2], [3, 4], [5, 6]]

In [30]:
temp = cf.items('url')
temp.pop(0)
temp

[('tmall', 'https://www.tmall.com/'),
 ('jingdong', 'https://channel.jd.com/women.html')]

In [4]:
profile = webdriver.FirefoxProfile('/home/ml/.mozilla/firefox/0k5sh6ex.default')
driver = webdriver.Firefox(profile)

In [ ]:
key_words = '羽绒服'

final = pd.DataFrame()
for key,value in best_seller_url.items():
    url = value
    url = url.replace(re.findall('&q=(.*?)&',url)[0],key_words)
    print(url)
    platform_parse = key+'_parse_links'
    items_list = cf.options(platform_parse)
    items_list.pop(0)
    items_list.pop(0)
    
    driver.get(url)
    data = pd.DataFrame([],columns=items_list)
    # 等待信息装载
    for i in range(0, 40):
        driver.execute_script("document.documentElement.scrollTop="+str(i*100 + 100))
        time.sleep(0.1)
            
    # 提取信息
    while True:
        try:
            for item in items_list:
                if 'attribute' in item:
                    temp = cf.get(platform_parse, item).split('|')
                    a = driver.find_elements_by_xpath(temp[0])
                    data[item] = pd.Series([a[i].get_attribute(temp[1]) for i in range(0, len(a))])
                else:
                    a = driver.find_elements_by_xpath(cf.get(platform_parse, item))
                    data[item] = pd.Series([a[i].text for i in range(0, len(a))])
            data['id'] = pd.Series([str(uuid.uuid4()) for i in range(0, len(a))])
        except:
            continue
        break
        
    final = final.append(data).reset_index(drop=True)

final['key_words'] = key_words
final.to_csv('best_seller.csv', index=False)

In [9]:
key_words = '羽绒服'

current_url = ''

platform_parse = '17_parse_links'
items_list = cf.options(platform_parse)
items_list.pop(0)
items_list.pop(0)

url = source_goods_url['17']
# url = url.replace(re.findall('?so=(.*?)&',url)[0],key_words)
current_url = url
driver.get(url)
page_nums = int(driver.find_elements_by_xpath(cf.get(platform_parse,'page_num')[:-1])[0].text.split(cf.get(platform_parse,'page_num')[-1])[1])
page_nums = 20
final = pd.DataFrame([],columns=items_list)
data = pd.DataFrame([],columns=items_list)
for page_num in range(0,page_nums):
    # 等待信息装载
    while True:                   
        result = [len(driver.find_elements_by_xpath(cf.get(platform_parse, item))) for item in items_list]
        if (0 not in result)&(len(set(result)) == 1):
            for i in range(0, 50):
                driver.execute_script("document.documentElement.scrollTop="+str(i*200 + 200))
                time.sleep(0.05)
            break

    # 提取信息
    while True:
        try:
            for item in items_list:
                if 'attribute' in item:
                    temp = cf.get(platform_parse, item).split('|')
                    a = driver.find_elements_by_xpath(temp[0])
                    data[item] = pd.Series([a[i].get_attribute(temp[1]) for i in range(0, len(a))])
                else:
                    a = driver.find_elements_by_xpath(cf.get(platform_parse, item))
                    data[item] = pd.Series([a[i].text for i in range(0, len(a))])
            data['id'] = pd.Series([str(uuid.uuid4()) for i in range(0, len(a))])
            data['page_num'] = page_num
        except:
            continue
        break

    # 最后一页退出
    if page_num == (page_nums-1):
        break

    # 翻页,并等待页面刷新
    driver.find_elements_by_xpath(cf.get(platform_parse, 'click'))[0].click()
    while current_url == driver.current_url:
        pass

    # 超过一半重复退出
    current_num = len(final)
    final = final.append(data).drop_duplicates(items_list).reset_index(drop=True)
    if len(final) < current_num + int(len(data)/2):
        break
#     final.to_sql('links', con=engine,  if_exists='append', index=False)

final['key_words'] = key_words
final.to_csv('source_goods.csv', index=False)

[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
149.45349645614624


In [10]:
import requests
from io import BytesIO
from PIL import Image

items = pd.read_csv('best_seller.csv')
for i in range(0,len(items)):
    try:
        if 'http' not in items['images_url_attribute'][i]:
            src = 'https:'+items['images_url_attribute'][i]
        else:
            src = items['images_url_attribute'][i]
        dir_path = 'data/best_seller/'+items['id'][i]+'.jpg'
        print(src)
        pic = Image.open(BytesIO(requests.get(src).content))
        pic = pic.convert('RGB')
        w, h = pic.size
        pic = pic.resize((int(w / 2), int(h / 2)), Image.ANTIALIAS)
        pic.save(dir_path)
    except requests.exceptions.ConnectionError:
        print('Sorrry,image cannot downloaded, url is error{}.'.format(src))

https://g-search3.alicdn.com/img/bao/uploaded/i4/i2/158748311/O1CN01pseu0r2BGUKYctcdM_!!0-item_pic.jpg
https://g-search2.alicdn.com/img/bao/uploaded/i4/i1/3008668235/O1CN011cPV0W2AhgDluySsK_!!0-item_pic.jpg
https://g-search3.alicdn.com/img/bao/uploaded/i4/i3/3477321991/O1CN01SMH0Xk1QZvBZ2rStt_!!0-item_pic.jpg
https://g-search1.alicdn.com/img/bao/uploaded/i4/i2/1809139068/O1CN01W781Py2GrC9E0vQV8_!!0-item_pic.jpg
https://g-search3.alicdn.com/img/bao/uploaded/i4/i4/2261867204/O1CN01hloSKO235TstbS3A0_!!0-item_pic.jpg
https://g-search2.alicdn.com/img/bao/uploaded/i4/i2/504429712/O1CN01VsgNpd2Lc9DODbIm7_!!504429712-0-lubanu-s.jpg
https://g-search1.alicdn.com/img/bao/uploaded/i4/i2/2206626886297/O1CN01DC3mTa1wO4dlJUF17_!!0-item_pic.jpg
https://g-search2.alicdn.com/img/bao/uploaded/i4/i1/4021071924/O1CN0126xwrS1Q5EeZ73A4T_!!0-item_pic.jpg
https://g-search3.alicdn.com/img/bao/uploaded/i4/i3/1689220258/O1CN01KKfwPI1DmCoX0gMEL_!!0-item_pic.jpg
https://g-search3.alicdn.com/img/bao/uploaded/i4/i2/2

https://img.alicdn.com/bao/uploaded/i2/3920909371/O1CN011vf96g2J5y9cSYld7_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i1/3646419379/O1CN01L2wu1c2J9dJbSAtt4_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i3/3972270038/O1CN01ka9gGI1C9Rflamx7n_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i3/1691787630/O1CN01slWLh626Eabl9e7CE_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i1/925814798/O1CN014wltl71lJWwzLj3v0_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i2/112394247/O1CN016tJFu31hFAiYySs80_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i4/2207358888682/O1CN01EqzwXI2E0PGpIYgYg_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i3/188124207/O1CN01VdUwsT1gwqsKyOq40_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i2/2201204126394/O1CN01OSOLMS1x6V34dwpWo_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i3/758103624/O1CN01CRK3as1cdpy3kImam_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i1/407916093/O1CNA1640Vxc1usdrBfj3Bf_!!407916093-0-p

In [11]:
items = pd.read_csv('source_goods.csv')
for i in range(0,len(items)):
    try:
        if 'http' not in items['images_url_attribute'][i]:
            src = 'https:'+items['images_url_attribute'][i]
        else:
            src = items['images_url_attribute'][i]
        dir_path = 'data/source_goods/'+items['id'][i]+'.jpg'
        print(src)
        pic = Image.open(BytesIO(requests.get(src).content))
        pic = pic.convert('RGB')
        w, h = pic.size
        pic = pic.resize((int(w / 2), int(h / 2)), Image.ANTIALIAS)
        pic.save(dir_path)
    except requests.exceptions.ConnectionError:
        print('Sorrry,image cannot downloaded, url is error{}.'.format(src))

https://img.alicdn.com/bao/uploaded/i4/1953685370/TB1tRtlo8TH8KJjy0FiXXcRsXXa_!!0-item_pic.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/1608188013/O1CN01qXA2lz2940FViwuQR_!!1608188013.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/514012/i1/fc847e79-5604-4609-8f5b-fd69c6aaa37d.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/461365/i1/9c2f5a04-27d3-4a9c-8cee-2f142db342da.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/1693111180/O1CN01iSVGGr1KaTyxJSZns_!!1693111180.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/219270/i1/6f3a02da-8584-4d52-b8b7-0d30901e4417.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/1128191467/O1CN01geeRLr1MhvcHpBaLQ_!!1128191467.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/1704646334/O1CN01A4j4lJ1wf1FReOvcI_!!1704646334.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/1607130301/O1CN01cs3ntg1E5tq6ZINHF_!!1607130301.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/225588/i1/95dbdce5-a6bc-4903-9a5f-9981c65

https://img01-gms.17zwd.com/imgextra/61149207/i1/21ae4740-a52d-4042-843b-a8b9e1dce967.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/514012/i1/8f879b62-a915-43f8-90a4-44fd00fa2740.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/461365/i1/d4a2d1e3-fe9a-4d84-b1d1-c4807c579499.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/219270/i1/48785a68-2f1e-4922-a619-c3e98cf876fd.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/219270/i1/86e0b179-420e-4829-920f-030401042731.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/219270/i1/6f3a02da-8584-4d52-b8b7-0d30901e4417.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61438566/i1/7eb9da3d-ca01-4097-9468-90e254f0e666.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61149207/i1/0171c425-f46b-44a0-8a4f-130cdb2ea5d9.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/437169/i1/a42d5a8e-8769-4a84-a4db-a47eb6fa834b.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61149207/i1/11d96ac8-6e29-4bfe-921c-a74a63023728.jpg_600x

https://img01-gms.17zwd.com/imgextra/61149207/i1/61745346-6097-43ec-9bcb-cf62e486202d.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/219270/i1/94c3fce6-f311-49e9-b9ea-2f69f5ea6110.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/62056866/i1/36b4180d-9776-4a6c-b8cf-067bb09176f4.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/219270/i1/1d836333-b7af-45bc-93a7-6bac8d3fc72d.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/219270/i1/3d611a1d-ca27-458b-8ba7-66477654cdd7.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/461365/i1/6d77821a-9f07-46b3-bae6-8cf23f7e0fad.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/292143129/O1CN01afPwHx1Yz7utae2Bt_!!292143129.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/228429/i1/0461ca77-d179-4dd6-aab2-af2f15b6b034.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/435616/i1/2cdd97ce-79c2-427b-b24a-f07c698bcd28.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/453558/i1/8a70c53a-05b6-41d2-a6d0-e494f3dac024.jpg_600x600

https://img01-gms.17zwd.com/imgextra/62056866/i1/eefa0c43-5294-4cbe-880d-1590ee4d890f.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/2047208536/O1CN01CWBweV2CvXRfiZ2jI_!!2047208536.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/228429/i1/9a4ccd1c-c66f-4acf-b89b-329ee9e02611.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/435616/i1/3331d5ad-1af7-4ff2-bf4d-069855a29949.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/62015702/i1/f6d4021d-2092-4dd1-9270-35666312cb54.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/2224599081/O1CN01FysYRX2Gx9InQXupR_!!2224599081.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/461365/i1/8964fc1c-1cb3-454e-b80b-fe9ed0546ca9.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/437169/i1/1041a9eb-de44-4336-bd69-1a7b921d01ee.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/298374/i1/da4a3bfe-ca36-433e-b502-cb63c391e162.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/292143129/O1CN01DEe4zK1Yz7v09VxbQ_!!292143129.jpg_6

https://img.alicdn.com/bao/uploaded/i4/756766871/O1CN01zb37zx20cxzldf7yw_!!756766871.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/324440050/O1CN014PVlF71CEwQpLNnGR_!!324440050.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/696084441/O1CN01QJmIDU1ig1YXJyodA_!!696084441.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/2873281417/O1CN01YoCdvW1ML1qxG5Np2_!!2873281417.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/24210684/O1CN01a0gIdN1GvJXMZo9ib_!!24210684.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/2647697205/O1CN01H2idJ7235wHW4jQwZ_!!2647697205.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61408274/i1/e4d56ad9-7b91-4ada-a410-68d185b8509a.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/1115551707/O1CN01CHIPUX1OTqiE712oq_!!1115551707.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/1115551707/O1CN01Zyeve81OTqiBE0lVR_!!1115551707.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/1941849010/O1CN01qGsV2U2GQdBGBsqM7_!!19418490

...

https://img.alicdn.com/bao/uploaded/i4/2627596085/O1CN01orPdJh1uoygbxuWFh_!!2627596085.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/514012/i1/884818b2-6185-45a1-a324-59f3d8192a00.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/292143129/O1CN01184xHD1Yz7unKGRuL_!!292143129.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/2627596085/O1CN01UasxCt1uoygXpyTyX_!!2627596085.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/175639/i1/03c67e03-2382-47d1-8e9c-2c2854ef958e.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/461365/i1/7d94abdf-66f5-428b-9a1c-fa042352ccc2.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/437169/i1/fdbb5d4d-9027-45ed-8703-31e71ada1683.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/514012/i1/9d7f4588-6c4d-41fa-9fbc-992824e91d97.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/435616/i1/9df190d7-3288-421c-900b-008b34048f3b.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/2230320610/O1CN01FrOfEJ1GNQDrakyJE_!!2230320610.jpg_60

https://img.alicdn.com/bao/uploaded/i3/2983430201/O1CN01vIxtvy1DM6EXFA9rp_!!2983430201.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/2627596085/O1CN01G0sDRi1uoygVwq4h1_!!2627596085.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/684741871/O1CN01D9bjy21PgxfE7Awy4_!!684741871.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61438566/i1/ff431bbc-a10e-4095-9a8b-e7560a87ab49.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/1744833927/O1CN01PotH401esbySQI6Bt_!!1744833927.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/684741871/O1CN01bhLEgX1Pgxf5JBcb5_!!684741871.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/684741871/O1CN01zijjr41PgxfHprTqD_!!684741871.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/437169/i1/20a47590-09a8-4465-945a-b83416337f1a.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/2627596085/O1CN017IfuFG1uoygaoVPM9_!!2627596085.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/226348/i1/9cdb5559-2b17-4bdb-b909-e7cacfc98de4

https://img.alicdn.com/bao/uploaded/i3/2983430201/O1CN01I80XpE1DM6EZosZMy_!!2983430201.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/1860166815/O1CN01cuzJh520DJoS0cAeY_!!1860166815.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/1637493512/O1CN011YX7Rr1boXbb2WASO_!!1637493512.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/199130/i1/b7f01e46-ced0-4014-b95d-fb07d8e09f62.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/2124567934/O1CN01SOntnu28Tp0jbRI0j_!!2124567934.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/1075277166/O1CN01cDZgyH22o4pYfGAGM_!!1075277166.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/199130/i1/44dd601d-4c2a-4663-88e4-da8869a4c989.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/2566575618/O1CN01z6Xd3u1rN5iqJACyE_!!2566575618.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/1580700661/O1CN011nTMEJ1GkmQMVTfCT_!!1580700661.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/2124567934/O1CN01NPXPyt28Tp0nEhD9n_!!212

https://img01-gms.17zwd.com/imgextra/227440/i1/309f0860-43da-494d-8c01-7dee1015f6fa.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/394104735/O1CN01VZe81y1kqfzfMX4xw_!!394104735.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/203143/i1/bb20abbf-c240-419a-bbd1-7b5fa9c3dd57.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/203143/i1/9bd51c04-e47b-4a4f-b1af-55330458ca7e.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61149207/i1/a5459229-3681-4561-a715-88e630cae2b9.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/1036822015/O1CN01bqeKaT1Qkuh7xpddm_!!1036822015.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/67025749/O1CN01HVzz0M1sL5bdL5PBR_!!67025749.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/891843764/O1CN01Bb2rOL1dfxPpzMvvt_!!891843764.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/437169/i1/ded9b459-8fc6-4920-a8d4-64a7619ac913.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/437169/i1/4a7d6244-55cb-4cfd-a070-7c4b89b7ce0a.jpg_600x60

https://img01-gms.17zwd.com/imgextra/227440/i1/a55da114-2d12-49d4-b43d-7a9c23e108d3.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/1086543704/O1CN01x7250W1dETeQl69lF-1086543704.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/581200435/O1CN01A2JeHB1F5GuiZkaOl-581200435.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/1643947056/O1CN01ubVQQW21zhGTWyIGr_!!1643947056.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/219270/i1/3e3ddf64-fdfc-4066-8f58-7d7957f22ca7.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61438566/i1/075c76ff-0c20-4529-8e5b-ed52c0c9b295.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61298223/i1/41435c8b-841e-4cd1-b84c-01f90e7165d2.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/2983430201/O1CN01ktAUem1DM6ENj3Wf2_!!2983430201.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/205832/i1/82e93719-3e40-43ba-b52f-c54184a22850.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/219270/i1/e2fca868-2da7-4cc0-975b-a81598116efe.jpg_60